# Statistics on basic predictors

We are going to limit ourselves to a few basic predictors and later try more advanced techniqes on these basic ones.

The predictors to be used are: DecisionTree, SVM and LogisticRegression.

here we are going to train each of the above predictors and show it's accuracy on a test dataset.

In [14]:
from Classifiers.classifiers import ready_training_data
from Utilities.orginizers import LearningData
from sklearn.model_selection import KFold, cross_val_score
from itertools import product
from random import sample
from multiprocessing import Pool
import matplotlib.pyplot as plt
import Utilities

pool = Pool()
ld = LearningData()

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklear

In [10]:
stock_names = sample(ld.get_stock_names(), 5)
stocks = [ready_training_data(sn) for sn in stock_names]

In [11]:
def run_model(model):
    scores = {sn: [] for sn in stock_names}
    validation_rounds = 3
    for i in range(validation_rounds):
        for sn, (data, classes) in zip(stock_names, stocks):
            scores[sn].append(sum(cross_val_score(model, data, classes))/3) 
    for sn in scores.keys():
        scores[sn] = sum(scores[sn])/validation_rounds
    return scores

### Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier

# create iterables for all options and use itertools product
criterion=['gini', 'entropy']
max_depth=[10, 20, 50, None]
# using percentages
min_samples_leaf=[0.005, 0.01, 0.02, 0.05, 0.1]

models = [DecisionTreeClassifier(criterion=c, max_depth=md, min_samples_leaf=msl) 
          for c, md, msl in product(criterion, max_depth, min_samples_leaf)]

accuracies = pool.map(run_model, models)


/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklear

In [13]:
print(accuracies)

[{'CYBX': 1.0, 'TRNX': 0.92940125111706884, 'Q': 0.99641147297215993, 'AIZ': 0.68064459790858622, 'PTX': 0.59286442405708462}, {'CYBX': 1.0, 'TRNX': 0.97319034852546915, 'Q': 0.99641147297215993, 'AIZ': 0.98792136943702591, 'PTX': 0.5938837920489296}, {'CYBX': 0.99444444444444446, 'TRNX': 0.98391420911528149, 'Q': 0.99641147297215993, 'AIZ': 0.98792136943702591, 'PTX': 0.5943594971117907}, {'CYBX': 0.98323917137476469, 'TRNX': 0.98391420911528149, 'Q': 0.99641147297215993, 'AIZ': 0.98792136943702591, 'PTX': 0.65688073394495416}, {'CYBX': 0.98323917137476469, 'TRNX': 0.98391420911528149, 'Q': 0.99641147297215993, 'AIZ': 0.98792136943702591, 'PTX': 0.71151885830784911}, {'CYBX': 1.0, 'TRNX': 0.92999702114983618, 'Q': 0.99641147297215993, 'AIZ': 0.68064459790858622, 'PTX': 0.59599048589874282}, {'CYBX': 1.0, 'TRNX': 0.97319034852546915, 'Q': 0.99641147297215993, 'AIZ': 0.98792136943702591, 'PTX': 0.59537886510363569}, {'CYBX': 0.99444444444444446, 'TRNX': 0.98391420911528149, 'Q': 0.99641

/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/parallel.py:540: UserWarning: Multiprocessing-backed parallel loops cannot be nested, setting n_jobs=1
  **self._backend_args)
/usr/local/lib/python3.5/dist-packages/sklear